<img src="files/Pics/LOGOS.png" width="800">

# Labor 04: Regulariált Logisztikus Regresszió (NemLineáris eset)
### Microchip Anomália:

A feladatunk meghatározni, hogy a mikrocsippek közül a mérési eredmények alapján melyik hibás és melyik nem.

Jelen gyakorlat során nem lineárisan szeparálható adathalmazzal fogunk dolgozni. A klasszifikációhoz szeretnénk használni a logisztikus regressziót, így a nem lineáris esetet bővítjük több tulajdonság (feature) bevezetésével (polinomiális regresszió).

Ebben a feladatban kipróbálásra kerülnek különböző regularizációs paraméterek, hogy jobban megérthessük hogyan működik a regularizálás (büntetés) és miként használható a túltanulás (overfit) megelőzésére. Figyeljük meg a változásokat a döntési határon, ahogy a lamdbát változtatjuk majd. Egy kis lambdával észrevehető lesz, hogy a klaszifikáció során szinte nem hibázik, azonban cserébe nagyon bonyolult görbét kapunk. Ez nem egy jó döntési görbe, vegyük észre, hogy a (-0,25; 1,5) -öt elfogadja, ami egy inkorrekt döntésnek tűnik az adathalmazunk alapján.

Egy nagyobb lambda segítségével azt láthatjuk, hogy egy egyszerűbb döntési határ jön létre, ami nem követi annyira az egyes adatokat így ez alultanult (underfitted).

### 1: Importáljuk be a megfelelő csomagokat majd alakítsuk ki a megfelelő változókat (X,Y,m,n)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
data = pd.read_csv('Lab4data.txt', header = None).to_numpy()        # adatok beolvasása a data változóba NumPy tömbként
X = data[:,0:2]                                                     # X rendezése
m,n = X.shape                                                       # adatok száma / feature-k száma
Y = data[:,2].reshape(m,1)                                          # Y rendezése
del data                                                            # felesleges változó törlése

print('X:', X.shape)                    # adattömbök méretének / adatok számának / feature-k számának kiírása
print('Y:', Y.shape)
print('Adatok száma:',m)
print('Feature-ök száma:',n)

### 2: Szedjük ki az elemeket az alapján, hogy átmentek-e a vizsgálaton és rajzoltassuk is ki őket

In [ ]:
def plotData(X,Y):
    pos = []                                                        # jó elemek listája 
    neg = []                                                        # rossz elemek listája

    for i in range(0,Y.size):                                       # végig megyünk az Y elemein
        if Y[i]==1:                                                 # ha Y értéke 1, akkor X azonos sora megy a pos -ba
            pos.append(X[i,:])
        elif Y[i]==0:                                               # ha Y értéke 0, akkor X azonos sora megy a neg -be
            neg.append(X[i,:])

    pos = np.array(pos)                                             # pos legyen NumPy tömb
    neg = np.array(neg)                                             # neg legyen NumPy tömb

    plt.scatter(pos[:, 0], pos[:, 1], c="g", marker="o", label="OK")        # pos kirajzolása
    plt.scatter(neg[:, 0], neg[:, 1], c="r", marker="x", label="Not OK")    # neg kirajzolása
    plt.title("Training data")
    plt.xlabel("Test 1")
    plt.ylabel("Test 2")
    plt.legend()
    plt.show()

    return pos,neg                                                  # visszaadjuk pos és neg -et a későbbi használatra

pos, neg = plotData(X,Y)                                            # függvény meghívása az adatainkra

Láthatjuk, hogy az adataink nem szeparálhatók lineárisan. <br>
Mivel adataink mindkét változó szerint $[-1, 1]$ intervallumba esnek elég jó eloszlással, így az adathalmaz további normalizálást nem igényel.

### 3: Illeszkedések

A modell illeszkedését tekintve a mintákra 3 különböző esetet különíthetünk el.

- Alul illeszkedésről (Underfit vagy High Bias) akkor beszélünk, amikor a modell túl egyszerű így nagy hibát okoz, mind a tanító adatokon mind a teszt adatokon.

- Pont jó az illeszkedés, ha a tanító adatokon és a teszt adatokon is alacsony hibát kapunk. Ez azt jelenti a tanulás során sikerült a lényeges információt megtanulni, ami segítségével az új minták is kellően jól besorolhatóak.

- Túltanulásról (Overfit vagy High Variance) beszélünk, ha a tanítás során a modell specifikusan rátanult a tanító mintákra. Ez azt eredményezi, hogy a tanulás során nagyon kis hibaértéket ért el, viszont a tanító adatsorban nem szereplő minták besorolásakor a hiba nagy lesz. 

<img src="files/Pics/L04_Fittings.png" width="800">

Az Underfit és Overfit jelenségek egy tipikus tanulás lefutása során. A cél az lenne, hogy a tanulást azon a ponton állítsuk meg, amikor a validációs hiba a legkisebb. 

<img src="files/Pics/L04_BiasVariance.png" width="400">

Gyakorlatban a rendelkezésre álló adatainkat 3 csoportra szoktuk osztani, ha elég nagy mintaszámmal rendelkezünk.
- Training set   (~70%): A tanulás során használt adatok, a moddell súlyainak beállítására
- Validation set (~15%): A tanulás leállítására a megfelelő epochnál, hiperparaméter optimalizálás
- Teszt set      (~15%): Tesztelés független adatokon, metrikák meghatározása

Jelen példánk során kis adathalmazzal dolgozunk, így az adatok ilyen fajta szétdarabolásától eltekintünk. Az elméleti kintekintés segíti a megértést.

#### Módszerek az Underfit és Overfit esetek kezelésére:

Underfit esetén a modellünk túl egyszerű. Megfelelő kompenzáció lehet: <br>
- Változók (feature) számának növelése
- Komplexebb modell választása

Overfit esetén a modellünk túl specifikusan tanul rá a tanító adatokra. Megfelelő kompenzáció lehet: <br>
- Kevesebb bementei változó
- Egyszerűbb modell
- Büntetés (regularizáció) bevezetése

Büntetés (Regularizáció) esetén az irány elv a következő: Komplex modellből (pl.: több változó) kiindulva az algoritmusnak számos lehetősége van, ezzel az underfit jelenségét nagy valószínűséggel lekezeltük. A költség függvényt kibővítve pedig büntetjük, ha tól sok változót használ a modell. Ilyen formában megteremtjük az optimális feltételt a feladat megoldásáshoz szükséges legegyszerűbb modell kialakításához. 

### 4: Bemenetek bővítése

Micros chip teszt példánk során a két teszt eredményünk a bemeneti változóink. A kívánt feladat megoldásához több változóra lesz szükségünk. Ennek egy lehetséges módja a bementi változók növelése az eredeti változóink hatványaival. 

$x_1,\  x_2 \Rightarrow\ 1,\ x_1,\ x_2,\ x_1^2,\ x_1x_2,\ x_2^2,\ x_1^3,\ x_1^2x_2,\ x_1x_2^2,\ x_2^3$

A BIAS tagot is beleértve 3-dik hatványig bővítve a bementi változóinkat a kezdeti 2 voltozó helyett 10 változóval számolhatunk.

Hozzuk létre a mapFeature() függvényt, ami a fenti leképezést hajtja végre. 

In [ ]:
def mapFeature(X1,X2,deg):                            # létrehozunk egy függvényt, ami az 'X1' és 'X2' összeg
#######################################################
                                                        # kombinációját megalkotja (X1, X2, X1^2, X1X2, X2^2,..., X2^3)
                                                        # a meghatározott fokszámig (deg - alapesetben 3)

    
     
                                                        # maga az egyes kombinációk létrehozása
                                                        # és a függvény kimeneti mátrixába rakása
#######################################################            
    return out

In [ ]:
deg = 3
X=mapFeature(X[:,0], X[:,1], deg)                           # hozzuk létre az X bővített változatát
print(X.shape)

A bemeneti változók kibővítése után rátérhetünk a költség függvény kialakítására.

### 5: Költségfüggvény függvény és Gradiens módszer

Az aktivációs függvényünk a szigmoid függvény lesz, ahogy a korábbi laborok során megszokhattuk.

A költség függvényt pedig egy büntető taggal bővítjük, az alábbi képlet szerint:

$ C(w)=\frac{1}{2m}\sum_{i=1}^m(h_w(x^i)-y^i)^2+\lambda\sum_{\color{red}{j=1}}^nw_{\color{red}{j}}^2 $

, ahol <br> 
$ \lambda $ a büntetés mértékét beállító paraméter <br>
$ i $ a minták indexe 1-től indulva. $i = 1...n$ <br>
$ j $ a bemeneti változók indexe 0-tól indulva. $\color{red}{ j = 0...m}$ <br>

Amire figyelni kell, hogy a BIAS-t nem büntetjük, vagyis az $x_{\color{red}{0}} = 1 $ BIAShoz tartozó $ \color{red}{w_0}$ súlyt nem szabad a büntetés kiszámításánál figyelembe venni. 

$ \lambda $ értékét túl nagyra választva az Underfit esete előjöhet.  

Tekintsük át több változós esetben hogyan alakul a költség függvény kibővítve a regularizációs taggal és hogyan fog a gradiens módszerbe illeszkedni.

$ C(w)=-\frac{1}{m}\sum_{i=1}^{m}y^i\cdot log(h_w(x^i))+(1-y^i)\cdot log(1-h_w(x^i))+\frac{\lambda}{2m}\sum_{j=1}^nw_j^2 $

A deriválást megkönnyítő megfontolásból a regularizáló tag konstansában $ \lambda $ helyett $\frac{\lambda}{2m}$ szerepel.

Gradiens csökkentéses (Gradient Descent) módszer súlyfrissítési alapképlete:

$ w_j = w_j - \mu \color{blue}{\frac{\partial}{\partial w_j}C(w)}$

A költség függvény deriváltjának számolásakor a BIAS eset külön kell kezelnünk. Vizsgáljuk meg $w_0$ és $w_1$ esetén.

$ \color{blue}{\frac{\partial}{\partial w_0}C(w)}=\frac{1}{m}\sum_{i=1}^{m}(h_w(x^i)-y^i)\cdot x_0^i+{\color{red} 0}$

$ \color{blue}{\frac{\partial}{\partial w_j}C(w)}=\frac{1}{m}\sum_{i=1}^{m}(h_w(x^i)-y^i)\cdot x_j^i+\frac{\lambda}{m}w_j $

A hasonlóságokat kihasználva gondoljuk át hogyan lehetne egy függvényben kiszámolni mátrixműveletek segítségével a költség függvényt és a gradienst. <br>
Implementálja a costFunctionReg() függvényt.

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def costFunctionReg(w,X,Y,Lambda=1):
#######################################################    
    
    
                                                 # aktiváció
                                                 # egy alternatív súlymátrixba kinullázzuk a BIAShoz tartozó súlyt
                                                 # büntetés

                                                 # költségfüggvény

                                               # gradiens
#######################################################
    return C,grad

In [ ]:
init_w = np.zeros((X.shape[1],1))                   # kezdeti súlyok (tiszta 0)
C, grad =costFunctionReg(init_w,X,Y)                # költségfüggvény / gradiens számolás
print('Expected cost at initial weight (zeros): 0.693')
print('Calculated cost at initial weight (zeros): %.4f' % C)

A költség függvény és a gradiensek kiszámolásának sikeres implementálása után következhet a súlyok módosítás. <br>

Gradient Descent algoritmus súlymódosító alapképlete:

$w_j:=w_j-\mu\frac{\partial}{\partial w_j}C(w)$ 

Implementálja a gradientDescent() függvényt. Használja fel a costFunctionReg() függvényt és a függvényen belül mentse el minden epoch költségét!

In [ ]:
def gradientDescent(X,Y,w,learning_rate,num_iters,Lambda):                         
####################################################    
    

                                                                            # iterációszámig futtatjuk
        
                                                                            # súlyok frissítése
                                                                            # költségfüggvény történet elmentése

    
####################################################    
    return w, C_history

Vizsgáljuk meg miként hat a tanulási ráta (learning rate vagy $ \mu$) és a regularizációs paraméter ($\lambda $) a költségfüggvény alakulására.

In [ ]:
learning_rate = 1
epoch = 800
Lambda = 0.02

w, C_history = gradientDescent(X,Y,init_w,learning_rate,epoch,Lambda)      
print('\nRegularized weight:\n',w)

plt.plot(C_history,label = "C_history")    
plt.title("Cost function trough the iterations")
plt.xlabel("Iteration")
plt.ylabel("Cost function value")
plt.legend()
plt.show()

### 6: Vizualizáció

Rajzoljuk ki a döntési határvonalat az eredeti adatsorunkon.

In [ ]:
plt.scatter(pos[:, 0], pos[:, 1], c="g", marker="o", label="OK")            # eredeti pozitív eredmények rajzolása
plt.scatter(neg[:, 0], neg[:, 1], c="r", marker="x", label="Not OK")        # eredeti negatív eredmények rajzolása

u_vals = np.linspace(-1,1.,50)                                              # a határ berajzolásához 1 paraméter
v_vals = np.linspace(-1,1.,50)                                              # második paraméter
z=np.zeros((len(u_vals),len(v_vals)))                                       # eredményváltozó inicializálása

for i in range(len(u_vals)):                                                # végigmegyünk az u_vals és v_vals elemein
    for j in range(len(v_vals)):
        z[i,j] = mapFeature(u_vals[i],v_vals[j],deg) @ w                        

plt.contour(u_vals,v_vals,z.transpose(),0)                                  # kirajzoljuk contour plottal a döntési határt
plt.title("Decision boundary and the training data")
plt.xlabel("Exam 1 score")
plt.ylabel("Exam 2 score")
plt.legend(loc=0)
plt.show()

### 7: A predikció pontossága

In [ ]:
def classificationPrediction(w,X):
    pred = (sigmoid(X @ w) > 0.5)
    return ((np.sum(pred==Y)/m)*100)

acc=classificationPrediction(w,X)
print('\nAccuracy of the classification:',acc, '%')